In [69]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Date, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [61]:
import pandas as pd
import os

In [17]:
import json

with open('../../data/games.json') as json_file:
    data = json.load(json_file)

In [18]:
game = data[0]

In [19]:
game['id']

174430

In [20]:
game['name']

'Gloomhaven'

In [26]:
len(game['image'])

89

## Stats

In [21]:
game['stats']

{'usersrated': 34829,
 'average': 8.83196,
 'bayesaverage': 8.576,
 'stddev': 1.60752,
 'median': 0.0,
 'owned': 55995,
 'trading': 346,
 'wanting': 1417,
 'wishing': 14645,
 'numcomments': 6545,
 'numweights': 1494,
 'averageweight': 3.8253,
 'ranks': [{'id': '1',
   'name': 'boardgame',
   'friendlyname': 'Board Game Rank',
   'value': 1},
  {'id': '5496',
   'name': 'thematic',
   'friendlyname': 'Thematic Rank',
   'value': 1},
  {'id': '5497',
   'name': 'strategygames',
   'friendlyname': 'Strategy Game Rank',
   'value': 1}]}

In [22]:
game['designers']

['Isaac Childres']

In [32]:
maximum = 0
for game in data:
    if game['description']:
        if len(game['description']) > maximum:
            maximum = len(game['description'])

In [33]:
maximum = 0
for game in data:
    if game['image']:
        if len(game['image']) > maximum:
            maximum = len(game['image'])

In [42]:
name = ''
maximum = 0
for game in data:
    if game['stats']['usersrated']:
        if game['stats']['usersrated'] > maximum:
            maximum = game['stats']['usersrated']
            name = game['name']

In [44]:
maximum
name

'Catan'

In [37]:
len(data[25]['image'])

89

In [74]:
Base = declarative_base()

class Boardgame(Base):
    """ Defines the data model for the table `boardgames`. """

    __tablename__ = 'boardgames'

    game_id = Column(String(100), primary_key=True, unique=True, nullable=False)
    name = Column(String(100), unique=False, nullable=False)
    image = Column(String(150), unique=False, nullable=True)
    thumbnail = Column(String(100), unique=False, nullable=True)
    description = Column(Text, unique=False, nullable=True)
    year_published = Column(Date, unique=False, nullable=True)
    min_age = Column(Integer, unique=False, nullable=True)
    number_of_ratings = Column(Integer, unique=False, nullable=True)
    average_user_rating = Column(Float, unique=False, nullable=True)
    number_of_ratings_weight = Column(Integer, unique=False, nullable=True)
    average_user_rating_weight = Column(Float, unique=False, nullable=True)
    bayes_average = Column(Float, unique=False, nullable=True)
                       

    def __repr__(self):
        game_repr = f"<Boardgame(game_id={self.game_id}, name={self.name})>"
        return game_repr

In [66]:
# DataBase User Configurations
DB_HOST = None
DB_PORT = None
DB_USER = None
DB_PW = None

DB_DIALECT = 'mysql+pymysql'
SQLALCHEMY_DATABASE_URI = os.environ.get('SQLALCHEMY_DATABASE_URI')

# Logic for deciding what type of DB to create
if SQLALCHEMY_DATABASE_URI is not None:
    pass
elif DB_HOST is None:
    SQLALCHEMY_DATABASE_URI = 'sqlite:///sample_sqlite_db.db'
else:
    SQLALCHEMY_DATABASE_URI = f'{DB_DIALECT}://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DATABASE}'
            
print(SQLALCHEMY_DATABASE_URI)

sqlite:///sample_sqlite_db.db


In [71]:
# Define Engine String
engine_string = SQLALCHEMY_DATABASE_URI

# Define Engine
engine = create_engine(engine_string)

# Create database
Base.metadata.create_all(engine)

In [75]:
sample_game = data[0]
print(sample_game['id'])
print(sample_game['name'])

174430
Gloomhaven


In [78]:
# Establish session with database
Session = sessionmaker(bind=engine)
session = Session()

# Define a boardgame in the ORM
game = Boardgame(game_id=sample_game['id'], name=sample_game['name'] )

# Add game to session
session.add(game)
# Commit additions to database
session.commit()

In [82]:
# Delete all entries in database
session.execute('''DELETE FROM boardgames''')
session.commit()

# Close session
session.close()